In [8]:
!pip install lightgbm catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 15.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [5]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath('../src'))

In [10]:
import optuna
from optimize_params import xgb_objective, lgbm_objective, cat_objective

In [11]:
from data_loader import DiabetesLoader
from preprocessing import get_preprocessor, add_important_interaction

loader = DiabetesLoader('../data/raw/train.csv')
X_train, y_train, X_val, y_val = loader.get_data()
pipeline = get_preprocessor()
X_train_processed = pipeline.fit_transform(X_train)
important_cols = ['family_history_diabetes_1', 'Relative_Activity', 'Age_BMI_Risk', 'Chronic_Metabolic_Load', 'physical_activity_minutes_per_week', 'Diet_Activity_Score', 'age', 'Age_WHR_Risk', 'Lipid_Accumulation', 'triglycerides']
X_train_full = add_important_interaction(X_train_processed, important_cols)

Loading data from /app/data/raw/train.csv
Data Loaded. Shape: (700000, 25)
Train X: (560000, 24)  Train y: (560000,)
Val X:   (140000, 24)    Val y:   (140000,)
combining 9 cols
added 36 cols as interactions


In [12]:
optimal_cols =  ['age',
 'physical_activity_minutes_per_week',
 'heart_rate',
 'triglycerides',
 'Cholesterol_Ratio',
 'Diet_Activity_Score',
 'Relative_Activity',
 'Age_BMI_Risk',
 'Age_BP_Risk',
 'Age_WHR_Risk',
 'family_history_diabetes_1',
 'cardiovascular_history_1',
 'Relative_Activity_x_Diet_Activity_Score',
 'Age_BMI_Risk_x_Chronic_Metabolic_Load',
 'Age_BMI_Risk_x_Lipid_Accumulation',
 'Chronic_Metabolic_Load_x_age',
 'physical_activity_minutes_per_week_x_Diet_Activity_Score',
 'age_x_Age_WHR_Risk',
 'age_x_Lipid_Accumulation',
 'Age_WHR_Risk_x_Lipid_Accumulation']

X_train_optimal = X_train_full[optimal_cols]

In [17]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: xgb_objective(trial, ratio=0.6049, X=X_train_optimal, y=y_train), n_trials=30)

[I 2025-12-24 17:14:03,185] A new study created in memory with name: no-name-d5df61e3-b3ce-4edb-a115-0dd12ddede7a
[I 2025-12-24 17:14:21,891] Trial 0 finished with value: 0.7132098624655804 and parameters: {'xgb_n_estimators': 485, 'xgb_max_depth': 3, 'xgb_lr': 0.03307690878597566, 'xgb_min_child': 3, 'xgb_gamma': 0.47374708997630516, 'xgb_subsample': 0.6558152655441026}. Best is trial 0 with value: 0.7132098624655804.
[I 2025-12-24 17:14:37,249] Trial 1 finished with value: 0.7147269641938628 and parameters: {'xgb_n_estimators': 344, 'xgb_max_depth': 4, 'xgb_lr': 0.03487598987371712, 'xgb_min_child': 3, 'xgb_gamma': 0.16230735027018955, 'xgb_subsample': 0.6814755569049262}. Best is trial 1 with value: 0.7147269641938628.
[I 2025-12-24 17:14:50,002] Trial 2 finished with value: 0.7025479126984795 and parameters: {'xgb_n_estimators': 359, 'xgb_max_depth': 4, 'xgb_lr': 0.01193353986403808, 'xgb_min_child': 4, 'xgb_gamma': 0.008233596265394327, 'xgb_subsample': 0.7567139354545847}. Best i

In [18]:
print("Best xgb Found:")
print(study.best_params)

Best xgb Found:
{'xgb_n_estimators': 546, 'xgb_max_depth': 4, 'xgb_lr': 0.09522141307403631, 'xgb_min_child': 5, 'xgb_gamma': 0.2829267954406464, 'xgb_subsample': 0.8714391222089948}


In [19]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: lgbm_objective(trial, ratio=0.6049, X=X_train_optimal, y=y_train), n_trials=30)

[I 2025-12-24 17:28:17,725] A new study created in memory with name: no-name-b2b5a24b-99f0-493d-afdb-69ca47769800
[I 2025-12-24 17:28:38,739] Trial 0 finished with value: 0.7203745494310815 and parameters: {'lgbm_n_estimators': 258, 'lgbm_lr': 0.02827637545991763, 'lgbm_num_leaves': 40}. Best is trial 0 with value: 0.7203745494310815.
[I 2025-12-24 17:29:02,531] Trial 1 finished with value: 0.7226882404079155 and parameters: {'lgbm_n_estimators': 381, 'lgbm_lr': 0.09354787764619724, 'lgbm_num_leaves': 37}. Best is trial 1 with value: 0.7226882404079155.
[I 2025-12-24 17:29:28,988] Trial 2 finished with value: 0.7137826155370425 and parameters: {'lgbm_n_estimators': 246, 'lgbm_lr': 0.014689843661151284, 'lgbm_num_leaves': 38}. Best is trial 1 with value: 0.7226882404079155.
[I 2025-12-24 17:30:03,817] Trial 3 finished with value: 0.714306344482602 and parameters: {'lgbm_n_estimators': 410, 'lgbm_lr': 0.012431809196245426, 'lgbm_num_leaves': 24}. Best is trial 1 with value: 0.72268824040

: 

In [ ]:
print("Best lgbm Found:")
print(study.best_params)